# Data Imputation


**Anna-Carolina Haensch** \
University of Maryland \
LMU Munich

**Tian Lou** \
Ohio Education Research Center \
The Ohio State University

**Xiangyu Ren** \
New York University

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10666376.svg)](https://doi.org/10.5281/zenodo.10666376)

**This notebook is developed for the [Data Literacy and Evidence Building Executive Class](https://www.socialdatascience.umd.edu/data-literacy).**

**The "Syntucky" data, which is synthetic in nature, is exclusively designed for training exercises. It is not intended to derive meaningful insights or make determinations about real-world populations.**

## Goals

In this notebook, we will explore various strategies for handling missing data in statistical analyses, focusing on three primary methods: Unconditional Mean Imputation, Conditional Mean Imputation, and Multiple Imputation. At the end of this notebook, we will use the 2015 Syntucky cohort data to illustrate how to apply these methods in practice. We will also provide a few alternative methods for dealing with data missingness.

**The specific questions we seek to answer in this notebook are:**
1. How do different imputation methods impact the results in statistical analysis?
2. What are the strengths and weaknesses of each imputation method in various data scenarios?
3. How should we deal with data missingness in practice?

**After completing this notebook, you should be able to:**
1. Understand the principles and applications of Unconditional Mean Imputation, Conditional Mean Imputation, and Multiple Imputation.
2. Evaluate the implications of each imputation technique on data analysis results.
3. Identify appropriate contexts for applying each method.
4. Comprehend different types of data missingness.

## 1. Data Preparation

To effectively explore different imputation methods in this notebook, we will need additional Python libraries that are specialized in statistical analysis and imputation techniques, such as `mice`.

In [ ]:
#Load libraries

#Data analysis tools
import pandas as pd
import numpy as np

#Data visualization tools
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

#Data imputation tools
from statsmodels.imputation import mice #Multiple imputation libraries
from scipy.stats import sem
import statistics

#Visualization Style
#Adjust the graph display definitions 
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

#Set visualization style
sns.set(style = 'whitegrid')

We will still use the 2015 Syntucky cohort data to illustrate examples. Before running the code below, please change <font color='red'> **YOUR DATA DIRECTORY**</font> to your own file path. If your data file has a different name, please change it in the second line of code `df_2015 = pd.read_csv(data_directory + 'syntucky_cohort_2015.csv')`.

In [ ]:
#Define data folder directory in your computer
data_directory  = 'YOUR DATA DIRECTORY'

#Read in 2015 cohort data
df_2015 = pd.read_csv(data_directory + 'syntucky_cohort_2015.csv')

#Check the first five rows of the data if you need a refresher
df_2015.head()

### 1.1. Introduce Missingness to the Data (It's a simulation!)

In our analysis, we assume the role of creators: we will **artificially introduce missing data into our dataset**. This is a departure from typical scenarios where we encounter already incomplete datasets without knowledge of how the missingness occurred. **Our approach allows us to understand and simulate the various mechanisms of missing data.**

The code provided demonstrates this process in a structured manner. First, it divides the dataset by **gender**, creating separate subsets for men and women, since we want to look at the effects of two different missingness mechanisms. Next, we remove any observations with missing values in specific variables, such as `gender`, `year7_earnings`, and  `year6_earnings`. **Removing observations with missing values in these variables is a crucial step for creating a robust baseline dataset.** This ensures that the analysis starts with a clean and complete set of data, and is free from any inherent missingness. It is important for establishing a control group, which allows us to accurately assess the impact of differnt imputation methods later on. 

Finally, **we introduce missingness to men's `year7_earnings` based on their `year6_earnings`**. We assign men with earnings below the 75th percentile a higher probability of missing data, mimicking a scenario where low-income workers are more likely to have incomplete data due to factors like relocation. This process gives us a controlled environment to test and compare various imputation methods.

In [ ]:
# Step 1: Split the dataset based on gender into two separate datasets: men and women
men_dataset = df_2015[df_2015['gender'] == 'Male'].copy()[['year7_earnings', 'gender', 'year6_earnings']]
women_dataset = df_2015[df_2015['gender'] == 'Female'].copy()[['year7_earnings', 'gender', 'year6_earnings']]

# Step 2: Remove observations with missing data in 'year7_earnings', 'year6_earnings', and 'gender' columns
men_no_na = men_dataset.dropna()
women_no_na = women_dataset.dropna()

# Step 3: Prepare the datasets for imputation by introducing artificial missingness
# Step 3a: Calculate the 75th percentile of 'year6_earnings' in the men's dataset
men_no_na = men_no_na.copy()
year6_earnings_quantile_75 = men_no_na['year6_earnings'].quantile(0.75)
# Step 3b: Assign a probability of 0.5 to men whose earnings are below 75 percentile,
#          and a probability of 0.1 to men whose earnings are above 75 percentile,
#          simulating a scenario where lower earners are more likely to have missing data
men_no_na['below_75_percentile'] = (men_no_na['year6_earnings'] < year6_earnings_quantile_75) * 1
men_no_na['missing_probability'] = np.where(men_no_na['year6_earnings'] < year6_earnings_quantile_75, 0.5, 0.1)

# Step 3c: Introduce missing values to 'year7_earnings' based on the calculated probabilities
men_missing = men_no_na.copy()
np.random.seed(1) #Set a seed so that we get consistent results
men_missing['year7_earnings'] = np.where(
    np.random.uniform(0, 1, len(men_missing)) < men_missing['missing_probability'],
    np.nan,
    men_missing['year7_earnings']
)

#Check men's dataset with artificially introduced missingness in year7_earnings
men_missing.head()

Similarly, we can also introduce artificial missingness to women's dataset. **However, in this example, the missingness in `year7_earnings` is dependent on itself, as opposed to `year6_earnings`.** This will result in important differences in the performance of different imputation methods. Specifically, we first identify the 75th percentile of `year7_earnings` for the women dataset. Women with year 7 earnings below this threshold are assigned a 40% chance of their `year7_earnings` being missing, simulating a scenario where low-income workers are more likely to have missing earnings, due to family responsibilities like child-rearing. Conversely, women above this earnings threshold are given a lower probability (10%) of missingness, assuming women with higher incomes are more likely to continue working and have complete data. 

In [ ]:
# Step 4: Prepare women's dataset for modeling missing data
# Step 4a: Calculate the 75th percentile of 'year7_earnings' in the women's dataset
earnings_quantile_75 = women_no_na['year7_earnings'].quantile(0.75)

# Step 4b: Assign missing probabilities based on year 7 earnings. Women below the 75th percentile in 'year7_earnings' 
# are assumed to have a higher probability (40%) of missing data due to potential family commitments. Those above 
# this threshold are assigned a lower probability (10%), reflecting a tendency to continue working.
women_no_na = women_no_na.copy()
women_no_na['below_75_percentile'] = (women_no_na['year7_earnings'] < earnings_quantile_75) * 1
women_no_na['missing_probability'] = np.where(women_no_na['year7_earnings'] < earnings_quantile_75, 0.4, 0.1)

# Step 4c: Create a dataset with introduced missing values in 'year7_earnings' based on the assigned probabilities
women_missing = women_no_na.copy()
np.random.seed(1) #Set a seed so that we get consistent results
women_missing['year7_earnings'] = np.where(
    np.random.uniform(0, 1, len(women_missing)) < women_missing['missing_probability'],
    np.nan,
    women_missing['year7_earnings']
)

#Check women's dataset with artificially introduced missingness in year7_earnings
women_missing.head()

### 1.2. Check Data Missingness

The code below calculates the percentage of missing values in `year7_earnings` for both men's and women's datasets. This gives us insights about the extent of artificially introduced missing data for subsequent imputation analysis.

In [ ]:
# Calculate the percentage of missing values in 'year7_earnings' within the men's dataset.
# This metric helps in quantifying the extent of missing data generated artificially.
men_missing_percentage = men_missing['year7_earnings'].isnull().mean() * 100
print("Missing Percentage for imputed Men Dataset:")
print(round(men_missing_percentage, 1), '%')

#Calculate amount of missingness for women
women_missing_percentage = women_missing['year7_earnings'].isnull().mean() * 100
print("Missing Percentage for imputed Women Dataset:")
print(round(women_missing_percentage, 1), '%')

## 2. Imputation Methods

### 2.1. Unconditional Mean Imputation

(Unconditional) mean imputation is a straightforward method for handling missing data, where missing values in a dataset are replaced with the mean of the observed values. This method is simple to implement and retains the sample size. However, it has notable weaknesses. **Mean imputation can lead to biased estimates of variances and covariances, as it underestimates the variability in the data.** This approach also does not account for the relationships between variables, potentially leading to unrealistic data distributions. In general, mean imputation fails to take into account the uncertainty associated with the missing data, limiting its effectiveness in statistical analysis.

We now apply unconditional mean imputation to the men's dataset. First, we create a copy of the dataset to preserve the original data with missing year 7 earnings. We then calculate the mean of the observed `year7_earnings` and use it as the imputed value. A new column, 'imputed', is added to indicate which records have been imputed. The values of missing `year7_earnings` are then replaced with the calculated mean. Finally, for better visualization in subsequent plots, we trim the extreme values, keeping only the middle 90% of the data. This process helps us analyze the effects of mean imputation on the dataset while maintaining the integrity of the original data structure.

In [ ]:
# Copy the men's dataset with missing values to apply unconditional mean imputation.
men_unconditional_imp = men_missing.copy()

# Compute the average of 'year7_earnings' from the observed (non-missing) values in the dataset.
men_missing_mean_earnings = men_unconditional_imp['year7_earnings'].mean()

# Add a new column 'imputed' to flag if 'year7_earnings' data are imputed (1 if imputed, 0 otherwise).
men_unconditional_imp['imputed'] = men_unconditional_imp['year7_earnings'].isna() * 1

# Replace missing 'year7_earnings' values with the computed mean to perform the imputation.
men_unconditional_imp['year7_earnings'] = men_unconditional_imp['year7_earnings'].fillna(men_missing_mean_earnings)

# Only for improved visualization (!) in the next plots, trim extreme values from the data.
# This involves removing the lowest and highest 5% of 'year7_earnings' values.
bottom_percentile = np.percentile(men_unconditional_imp['year7_earnings'], 5)
top_percentile = np.percentile(men_unconditional_imp['year7_earnings'], 95)
men_unconditional_imp_trimmed = men_unconditional_imp[(men_unconditional_imp['year7_earnings'] >= bottom_percentile) &
                                                             (men_unconditional_imp['year7_earnings'] <= top_percentile)]

The scatter plot below visualizes the relationship between `year6_earnings` and `year7_earnings` in the men's dataset. Blue dots represent observed values where 'year7_earnings' were originally present, and red dots show imputed values, where `year7_earnings` were missing and have been filled in using the unconditional mean imputation method. This visualization aids in comparing the distribution and trends of both observed and imputed data, providing insights into the impact of the imputation process on the dataset's overall characteristics. Notice that the imputed red values are all on a horizontal line (since we only used the mean to fill in the gaps). Is it realistic that the original, but now missing values are all equal to the mean? No!

In [ ]:
# Generate a scatter plot for the observed and imputed data in the trimmed men's dataset
# Points represent earnings data where 'year7_earnings' are not imputed (original data)
men_observed = men_unconditional_imp_trimmed[men_unconditional_imp_trimmed['imputed'] == 0]

#Set plot size
plt.figure(figsize = (12, 7))

#Plot observed data
plt.scatter(men_observed['year6_earnings'], men_observed['year7_earnings'], 
            color = 'blue', label = 'Observed Data', s = 10)

# Points represent earnings data where 'year7_earnings' are imputed
men_only_unconditional_imp= men_unconditional_imp_trimmed[men_unconditional_imp_trimmed['imputed'] == 1]

#Plot imputed data
plt.scatter(men_only_unconditional_imp['year6_earnings'], men_only_unconditional_imp['year7_earnings'], 
            color = 'red', label = 'Imputed Data', s = 10)

# Enhance the plot with labels, legend, and a descriptive title
plt.xlabel('Year 6 Earnings')
plt.ylabel('Year 7 Earnings')
plt.legend()
plt.title('Comparison of Observed and Imputed Year 7 Earnings Relative to Year 6 Earnings for Men')
plt.show()

####  Reflect on the impact of mean imputation on various statistical measures in our dataset. 

- How does replacing missing values with the mean affect the overall mean and variance of 'year7_earnings'?

Below you can find code that creates mean and variance estimates for the full data (before we artificially deleted the data), the dataset with missing data (and a complete cases analysis, i.e. we did not include the observations with missing values) and the dataset with our mean imputed values.


In [ ]:
#Mean and variance of the data without any missing values
men_mean_full = round(np.mean(men_no_na['year7_earnings']), 1)
men_variance_full = round(np.var(men_no_na['year7_earnings']), 1)

#Calculate mean and variance for the men dataset with missing data 
men_mean_missing = round(np.mean(men_missing['year7_earnings']), 1)
men_variance_missing = round(np.var(men_missing['year7_earnings']), 1)

#Calculate mean and variance for the men dataset with imputed data 
men_mean_unconditional_imp = round(np.mean(men_unconditional_imp['year7_earnings']), 1)
men_variance_unconditional_imp = round(np.var(men_unconditional_imp['year7_earnings']), 1)

# Display the results
men_results = pd.DataFrame(data = {'Data': ['Full Men Dataset', 'Men Dataset with Missing Values', 'Men Dataset with Imputed Data'], 
                                   'Mean': [men_mean_full, men_mean_missing, men_mean_unconditional_imp],
                                   'Variance': [men_variance_full, men_variance_missing, men_variance_unconditional_imp]})

print(men_results)

Both the mean estimated with the men dataset with missing data and the one after imputation are way too high! And the variance after imputation is way too low (the imputed values have variance zero as they all equal the mean of the complete cases)! 

### 2.2 Conditional Mean Imputation

As a next step, let us adjust our imputations a little bit. Instead of using the overall mean, we now use the mean 'year7_earnings' calculated from the oberved data of men below the 75th percentile of 'year6_earnings' and the mean 'year7_earnings' for men from the oberved data of men above the 75th percentile of 'year6_earnings'.

In [ ]:
# Prepare conditional mean imputation for the men's dataset based on year6 earnings quantile.
men_conditional_imp = men_missing.copy()

# Flag the 'year7_earnings' data as imputed or not imputed.
men_conditional_imp['imputed'] = (men_conditional_imp['year7_earnings'].isna()) * 1

# Calculate mean 'year7_earnings' for men below the 75th percentile of 'year6_earnings'.
men_below_75_avg_earnings = men_conditional_imp[(men_conditional_imp['imputed'] == 0) & 
                                                    (men_conditional_imp['below_75_percentile'] == 1)]['year7_earnings'].mean()

# Calculate mean 'year7_earnings' for men above the 75th percentile of 'year6_earnings'.
men_above_75_avg_earnings = men_conditional_imp[(men_conditional_imp['imputed'] == 0) & 
                                                    (men_conditional_imp['below_75_percentile'] == 0)]['year7_earnings'].mean()

# Impute the calculated means based on whether individuals are below or above the 75th percentile in year 6.
men_conditional_imp.loc[(men_conditional_imp['below_75_percentile'] == 1) & (men_conditional_imp['year7_earnings'].isnull()), 'year7_earnings'] = men_below_75_avg_earnings
men_conditional_imp.loc[(men_conditional_imp['below_75_percentile'] == 0) & (men_conditional_imp['year7_earnings'].isnull()), 'year7_earnings'] = men_above_75_avg_earnings

# Trim the dataset for visualization purposes by removing the extreme 5% of 'year7_earnings' values.
bottom_percentile = np.percentile(men_conditional_imp['year7_earnings'], 5)
top_percentile = np.percentile(men_conditional_imp['year7_earnings'], 95)
men_conditional_imp_trimmed = men_conditional_imp[(men_conditional_imp['year7_earnings'] >= bottom_percentile) & (men_conditional_imp['year7_earnings'] <= top_percentile)]

Let us take a brief look at the two mean values:

In [ ]:
print(men_above_75_avg_earnings)
print(men_below_75_avg_earnings)

And the visualized imputed values. It is a little better but still not good. We now have some minimal variation in the imputed data, but it is still very unlikely that the original, but missing values all equal the means for our two groups.

In [ ]:
# Create a scatter plot for observed and imputed data in the trimmed conditional men's dataset.
# Original data where 'year7_earnings' are not imputed.
men_observed_conditional_imp = men_conditional_imp_trimmed[men_conditional_imp_trimmed['imputed'] == 0]

#Set plot size 
plt.figure(figsize = (12, 7))

#Plot observed data
plt.scatter(men_observed_conditional_imp['year6_earnings'], men_observed_conditional_imp['year7_earnings'], 
            color = 'blue', label = 'Observed Data', s = 10)

# Data where 'year7_earnings' values have been imputed.
men_only_conditional_imp= men_conditional_imp_trimmed[men_conditional_imp_trimmed['imputed'] == 1]

#Plot imputed data
plt.scatter(men_only_conditional_imp['year6_earnings'], men_only_conditional_imp['year7_earnings'], 
            color = 'red', label = 'Imputed Data', s = 10)

# Enhance the plot with labels, legend, and a descriptive title.
plt.xlabel('Year 6 Earnings')
plt.ylabel('Year 7 Earnings')
plt.legend()
plt.title('Year 6 vs. Year 7 Earnings: Observed vs. Imputed Data (Conditional Method) for Men')
plt.show()

#### Checkpoint 2: Reflect on the nuances of conditional imputation with these questions:

- How does conditional imputation, based on different quantiles of 'year6_earnings', affect the distribution of 'year7_earnings'? Consider differences in the means and variances within each quantile group.

Below you can find code that creates mean and variance estimates for the full data (before we artificially deleted the data), the dataset with missing data (and a complete cases analysis, i.e. we did not include the observations with missing values) and the dataset with our mean imputed values.

In [ ]:
# Calculate mean and variance for the men dataset with imputed data 
men_mean_conditional_imp = np.mean(men_conditional_imp['year7_earnings'])
men_variance_conditional_imp = np.var(men_conditional_imp['year7_earnings'])

# Display the results
print("Full Men Dataset - Mean:", np.mean(men_no_na['year7_earnings']), "Variance:", np.var(men_no_na['year7_earnings']))
print("Men Dataset with Missing Data - Mean:", np.mean(men_missing['year7_earnings']), "Variance:", np.var(men_missing['year7_earnings']))
print("Conditional Men Dataset with Imputed Data - Mean:", men_mean_conditional_imp, "Variance:", men_variance_conditional_imp)

### 2.3. Multiple imputation



Having explored conditional imputation, where we tailored our approach based on subsets of data, we now transition to **multiple imputation**. This advanced technique addresses some of the limitations of simpler methods by creating several different imputed datasets and combining them for analysis. Multiple imputation incorporates random variation, **reflecting the uncertainty inherent in imputing missing values**. This approach provides a more robust and comprehensive way to handle missing data, especially when the missingness is not random and is related to the observed data in complex ways.

Multiple imputation is a refined technique that builds upon the principles of conditional imputation. Like conditional imputation, it uses observed data to impute missing values. **However, it adds a layer of complexity and realism. In this method, each missing value is replaced not once, but multiple times, creating several complete datasets. Each of these datasets incorporates a random variation to mirror the variability in the original data and the inherent uncertainty in predicting unseen information.** This process yields a series of plausible data sets, each slightly different from the others, reflecting both the observed patterns and the randomness of real-world data. **The results from these multiple datasets are then combined, offering a more comprehensive and nuanced understanding than a single imputation could provide.**

As it is a little bit more complicated we need specialized functions to implement it. This code snippet below implements multiple imputation using the MICE (Multiple Imputation by Chained Equations) technique and applies Rubin's rules for combining results from multiple imputations for the men's dataset:

1) **Preparing for Imputation**: The code first identifies `year7_earnings` as the target column for imputation. It then copies the necessary columns from the men's dataset with missing data.

2) **Setting Up Multiple Imputations**: It defines the number of imputations (5 in this case) and initializes a process to store each imputed dataset.

3) **Using MICE for imputation**: MICE is used to impute missing values across several iterations, creating a series of datasets with imputed values.

4) **Analysis of Imputed Data**: Each imputed dataset is analyzed to calculate the mean and variance of 'year7_earnings'.

5) **Applying Rubin's Rules**: Rubin's rules (in this case taking the mean of the iterations, for standard errors the procedure would be more complicated) are used to combine the results from all imputed datasets to produce overall estimates for the mean and variance of 'year7_earnings'.

In [ ]:
# Select the column to be imputed ('year7_earnings') and prepare the dataset for multiple imputation
columns_for_imputation = ['year7_earnings']
men_missing_mi = men_missing[['year7_earnings','year6_earnings']].copy()

# Define the number of imputations to perform
num_imputations_men = 5

# Initialize a list to store the datasets generated from each imputation iteration
imputed_datasets_men = []

# Set up the MICE (Multiple Imputation by Chained Equations) model for multiple imputations
men_mice = mice.MICEData(men_missing_mi)

# Run multiple imputation, updating the dataset for each iteration
for i in range(num_imputations_men):
    men_mice.update_all()
    imputed_dataset_men = men_mice.data.copy()
    imputed_datasets_men.append(imputed_dataset_men)

# Analyze each imputed dataset, calculating the mean and variance of 'year7_earnings'
results_men = []
for imputed_data in imputed_datasets_men:
    mean_estimate = np.mean(imputed_data['year7_earnings'])
    variance_estimate = np.var(imputed_data['year7_earnings'])
    results_men.append({'Mean': mean_estimate, 'Variance': variance_estimate})

# Combine results from all imputations using Rubin's rules to obtain overall mean and variance estimates
mi_mean_estimate_men = np.mean([result['Mean'] for result in results_men])
mi_variance_estimate_men = np.mean([result['Variance'] for result in results_men]) 

# Display the combined results with confidence intervals
print(f"Combined Mean Estimate for men: {mi_mean_estimate_men:.2f}")
print(f"Combined Variance Estimate for men: {mi_variance_estimate_men:.2f}")

Let us take a look at the plot now! Feel free to ignore all this code fiddling:


In [ ]:
# Select the first imputed dataset
first_imputed_data = imputed_datasets_men[0]

# Original data where 'year7_earnings' are not imputed
men_observed_mi = men_missing.dropna()

# Reset the index of the men_missing
men_missing_reset = men_missing.reset_index(drop=True)

# Reset the index of the first imputed dataset
first_imputed_data_reset = first_imputed_data.reset_index(drop=True)

# Generate the missing_indices mask again after resetting the index
missing_indices_reset = men_missing_reset['year7_earnings'].isna()

# Use the missing_indices mask on the reset first imputed dataset
men_mi = first_imputed_data_reset[missing_indices_reset]


# Set plot size 
plt.figure(figsize=(12, 7))

# Plot observed data
plt.scatter(men_observed_mi['year6_earnings'], men_observed_mi['year7_earnings'], 
            color='blue', label='Observed Data', s=10)

# Plot imputed data
plt.scatter(men_mi['year6_earnings'], men_mi['year7_earnings'], 
            color='red', label='Imputed Data', s=10)

# Enhance the plot with labels, legend, and a descriptive title.
plt.xlabel('Year 6 Earnings')
plt.ylabel('Year 7 Earnings')
plt.legend()
plt.title('Year 6 vs. Year 7 Earnings: Observed vs. Imputed Data (One repition of Multiple Imputation) for Men')
plt.show()

#### Checkpoint 3: Multiple Imputation Results

Please take a look at the means and variances after multiple imputation: What do you notice in comparison to the simpler imputation appraoches?

In [ ]:
# The calculations of mean and variance of multiple imputation are in the previous codes snippet 
# Display the results
print("Full Men Dataset - Mean:", np.mean(men_no_na['year7_earnings']), "Variance:", np.var(men_no_na['year7_earnings']))
print("Men Dataset with Missing Data - Mean:", np.mean(men_missing['year7_earnings']), "Variance:", np.var(men_missing['year7_earnings']))
print("Multiple imputation Men Dataset with Imputed Data - Mean:", mi_mean_estimate_men, "Variance:", mi_variance_estimate_men)

## 3. All is well? Not always: Missing not at random

Do you remember that we created a second dataset with the women's data? And do you remember that the missingness among women was dependent on their year 7 earnings? In comparison to the men, where we modeled the missingness as dependent on the year6 earnings (i.e. a year before and a separate variable!). But what does that mean? Can we still use Multiple Imputation? Let us take a look: 

In [ ]:
# Identify the column we want to impute
columns_for_imputation = ['year7_earnings']
multi_women_missing = women_missing[['year7_earnings','year6_earnings']].copy()

# Define the number of imputation
num_imputations_women = 5
imputed_datasets_women = []

# Using MICE for multiple imputations
mice_women = mice.MICEData(multi_women_missing)

for i in range(num_imputations_women):
    mice_women.update_all()
    imputed_dataset_women = mice_women.data.copy()
    imputed_datasets_women.append(imputed_dataset_women)

# Perform analysis on each imputed dataset
results_women = []

for imputed_data in imputed_datasets_women:
    mean_estimate = np.mean(imputed_data['year7_earnings'])
    variance_estimate = np.var(imputed_data['year7_earnings'])

    results_women.append({'Mean': mean_estimate, 'Variance': variance_estimate})

# Combine results using Rubin's rules
mi_mean_estimate_women = np.mean([result['Mean'] for result in results_women])
mi_variance_estimate_women = np.mean([result['Variance'] for result in results_women]) 

women_mean_full = np.mean(women_no_na['year7_earnings'])
women_variance_full = np.var(women_no_na['year7_earnings'])

# Calculate mean and variance for the men dataset with missing data 
women_mean_missing = np.mean(women_missing['year7_earnings'])
women_variance_missing = np.var(women_missing['year7_earnings'])

# The calculations of mean and variance of multiple imputation are in the previous codes snippet 
# Display the results
print("Full Women Dataset - Mean:", women_mean_full, "Variance:", women_variance_full)
print("Women Dataset with Missing Data - Mean:", women_mean_missing, "Variance:", women_variance_missing)
print("Multiple Imputation Women Dataset with Imputed Data - Mean:", mi_mean_estimate_women, "Variance:", mi_variance_estimate_women)



Even after imputation, we still obeserve some bias! Why is that the case? Unfortunately our data here is what we call MNAR data. Missing Not At Random (MNAR) data occurs when the probability of a data point being missing is related to the missing value itself, creating a challenge in ensuring that analyses are not biased. This scenario poses a  problem for techniques like multiple imputation, which relies on patterns and relationships within the observed data to estimate missing values. However, in the case of MNAR, these patterns are incomplete or skewed because the missingness is inherently linked to unobserved values. Consequently, imputations made under MNAR can be systematically biased, as they may not accurately reflect the nature or distribution of the missing data. We do have a strong predictor here in the MI model of both missingness and the variable to be imputed, so the bias is not too bad. In general, this limitation of multiple imputation in the context of MNAR highlights the need for careful consideration and potentially even more advanced or tailored statistical methods (for example the Heckman selection model or a pattern-mixture model) to address missing data biases accurately.


## 4. Impute Year 7 Missing Earnings in the 2015 Synucky Cohort Data

In this section, we will use the 2015 Syntucky cohort data to illustrate how to apply the mean imputation and multiple imputation methods in practice. We will discuss the advantages and drawbacks of each method and then provide a few alternative methods to fill in missing data. 

### 4.1. Average Year 7 Earnings by Degree Type
Let's assume that academic advisors in Syntucky want to understand earnings by degree levels so that they can make recommendations to students. A data analyst uses the 2015 Syntucky cohort data to generate average year 7 earnings for associate’s and bachelor’s degree earners. The visualization below shows that during year 7, Bachelor's degree holders and Associate's degree holders have similar earnings. Based on this result, should academic advisors recommend students to pursue an Associate's degree, given its similar expected earnings, lower cost, and quicker completion time compared to a Bachelor's degree? One may argue that year 7 earnings are not an ideal measure, because it does not tell us future earnings growth and some graduates may not find suitable jobs immediately after college graduation. However, if this is the only measure we can use, are there any other limitations from the underlying data, especially the missingness in our data?

In [ ]:
# Limit the data to bachelor's and associate's degree holders
df_ba_aa = df_2015[df_2015['high_completion_label'].isin(["Bachelor", "Associate"])]

In [ ]:
#Create a visualization to show the average earnings of the 2015 cohort bachelor's and associate's degree holders

# Set plot size
# The first number is the width and the second number is the height.
fig, ax = plt.subplots(figsize = (8, 5))

#Create a bar chart to show average earnings
sns.barplot(df_ba_aa['high_completion_label'], 
            df_ba_aa['year7_earnings'], 
            order = ['Associate', 'Bachelor'])

# Change Y-axis and X-axis labels
# You can change the label size in `fontsize = `
ax.set_ylabel('Year 7 Earnings', fontsize = 12)
ax.set_xlabel('Highest Degree', fontsize = 12)
#Adjust y axis range
ax.set_ylim([0, 50000])

# Add values above bars
for i, v in enumerate([38973, 39246]):
    ax.text(i, v + 3000, f'${v:,.0f}', ha='center')

# Y-axis tick labels: Change the format of earnings displayed on Y-axis
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

# Data source
# The first two numbers indicate the coordinate (or (x,y) position) of the text
# You may need to adjust the numbers a few times to place it in an ideal place
plt.figtext(0.65, 0, 'Data Source: Syntucky Data')

# X-axis tick labels: 
# We can define x tick labels, such as using capital letters for the first letter of each word
ax.set_xticklabels(["Associate's Degree", "Bachelor's Degree"])

#Add a title
plt.title('Of the 2015 Syntucky Cohort, Associate’s degree holders \n and bachelor’s degree holders had similar year 7 earnings')

# Show the plot
plt.show()

### 4.2. Missing Rates in Year 7 Earnings by Subgroups

In this section, we calculate year 7 earnings missing rates by degree type, gender, and underrepresented miniorty (URM) status. Based on the summary table and the bar chart below, we can see that over half of bachelor's degree holders' year 7 earnings are missing, compared to 40% year 7 earnings missing rate for associate's degree holders. Moreover, across all subgroups we examine, bachelor's degree holders have higher year 7 earnings missing rates than associate's degree holders. The difference is especially large for non-URM students and male students. The two groups typically have higher average earnings than URM students and female students. 

Except for being unemployed during year 7, what could be the other reasons that a student's earning is missing in our data? First of all, the Synthetic data is only based on in-state Unemployment Insurance (UI) wage records. If a student moved out of state, his/her earnings would not show in our data. It's likely that Bachelor's degree students who expect higher earnings in other states are more likely to move. Second of all, even if a student did not move to other states, UI wage records does not cover all the jobs. Federal workers, self-employed, and contractor jobs are not included in the data. Lastly, even if a student's earnings showed in our data, what if their earnings are one dollar? In this case, should we treat this data point as missing data or exclude from our calculation?

In addition to the missingness in earnings data, there may be missingness in the higher education data as well. For example, some students may transfer to colleges in other states or overseas and receive degrees from out-of-state or international postsecondary education institutions. These students would not even be included in our sample. For another example, some students may have received additional training, such as online degrees and certificates or apprenticeship training, which could lead to higher earnings. Our current data does not allow us to separate students who received additional trainings from those who did not. Finally, different institutions may document degree information or the changes in degree information differently, which may lead to misclassification of degree types.

In summary, it is unlikely that year 7 earnings are missing complete at random, because some groups, especially the groups that are more likely to have higher earnings have higher missing rates. What about missing at random and missing not at random? As dicussed in the previous section, it is impossible to separate or detect these two types of missingness. However, given the potential reasons for data missingness listed above, it is likely that both types of missingness exist in our data.

In [ ]:
#Calculate year 7 earnings missing rates by subgroups

#Generate a year 7 earnings missing indicator
df_ba_aa.loc[:, 'y7_missing'] = df_ba_aa['year7_earnings'].isna() * 1

#Calculate missing rate by degree type
missing = df_ba_aa.groupby(['high_completion_label'])['y7_missing'].agg(['mean']).reset_index()

#Add a category indicator
missing['cat'] = 'Full Sample'

#Calculate missing rate by degree type and URM status
temp_df = df_ba_aa.groupby(['high_completion_label', 'urm_status'])['y7_missing'].agg(['mean']).reset_index().rename(columns = {'urm_status' : 'cat'})

#Combine the DataFrames
missing = pd.concat([missing, temp_df])

#Calculate missing rate by degree type and gender
temp_df = df_ba_aa.groupby(['high_completion_label', 'gender'])['y7_missing'].agg(['mean']).reset_index().rename(columns = {'gender' : 'cat'})

#Combine the DataFrames
missing = pd.concat([missing, temp_df])

#Check the results
missing

In [ ]:
#Create a bar plot to show the differences between year 7 earnings missing rates by subgroups

# Set plot size
# The first number is the width and the second number is the height.
fig, ax = plt.subplots(figsize = (8, 5))

#Create the bar plot
sns.barplot(x = missing['cat'], 
            y = missing['mean'], 
            hue = missing['high_completion_label'])

# Change Y-axis and X-axis labels
# You can change the label size in `fontsize = `
ax.set_ylabel('Missing Rate', fontsize = 12)
ax.set_xlabel('Category', fontsize = 12)
#Adjust y axis range
ax.set_ylim([0, 0.8])

# Data source
# The first two numbers indicate the coordinate (or (x,y) position) of the text
# You may need to adjust the numbers a few times to place it in an ideal place
plt.figtext(0.65, 0, 'Data Source: Syntucky Data')

#Adjust legend
ax.legend(title = 'Highest Degree Level')

#Add a title
plt.title('Of the 2015 Syntucky Cohort, Bachelor’s degree holders Have Higher \n Year 7 Earnings Missing Rates Than Associate’s Degree Holders' )

# Show the plot
plt.show()

### 4.3. Impute Year 7 Missing Earnings with Mean Imputation and Multiple Imputation

In this section, we apply the unconditional and conditional mean imputation approach, as well as the multiple imputation method and examine how the results differ. In the unconditional mean imputation, we fill in missing data with average year 7 earnings for bachelor's and associate's degree holders separately. If we are doing conditional mean imputation, we calculate average year 7 earnings by degree type, gender, and URM status. Then we fill in missing data for students in each subgroup. Finally, with  multiple imputation, we still use the `mice` library and repeat the process 50 times. 

The bar chart below shows the results from different imputation methods. We can see that unconditional mean imputation did not change the results at all and conditional mean imputation increased the earnings difference between bachelor's and associate's degree holders slightly. The multiple imputation method increased the earnings difference even further. However, when comparing our results with other data sources, such as the expected earnings by educational attainment on the Bureau of Labor Statistics website or National Center of Education Statistics website, the earnings difference we find with the Syntucky data is way smaller.

While the multiple imputation method improved our results and it is possible that associate's degree holders have similar expected earnings to bachelor's degree holders in Syntucky, we still need to keep the potential data missingness in mind when interpreting our results. Moreover, the imputation method is still imited by our data. Since the missingness in our data is likely to be MNAR, the multiple imputation method cannot information that's not included in our data or correct the missingness.

In [ ]:
#Average year 7 earnings by degree type
avg_y7_earn_degr_df = df_ba_aa.groupby(['high_completion_label'])['year7_earnings'].agg(['mean']).reset_index().rename(columns = {'mean' : 'uncon_mean'})

#Average year 7 earnings by degree type and subgroup
avg_y7_earn_degr_subg_df = df_ba_aa.groupby(['high_completion_label', 'gender', 'urm_status'])['year7_earnings'].agg(['mean']).reset_index().rename(columns = {'mean' : 'con_mean'})

#Save the original data to a new DataFrame for data imputation
df_mean_impute = df_ba_aa[['high_completion_label', 'gender', 'urm_status', 'year7_earnings']]

#Merge DataFrames
df_mean_impute = df_mean_impute.merge(avg_y7_earn_degr_df, on = 'high_completion_label', how = 'left')
df_mean_impute = df_mean_impute.merge(avg_y7_earn_degr_subg_df, on = ['high_completion_label', 'gender', 'urm_status'], how = 'left')

#Mean imputation
conditions = [df_mean_impute['year7_earnings'].isnull() == True]

uncon_choices = [df_mean_impute['uncon_mean']]
con_choices = [df_mean_impute['con_mean']]

df_mean_impute['year7_earnings_uncon'] = np.select(conditions, uncon_choices, default = df_mean_impute['year7_earnings'])
df_mean_impute['year7_earnings_con'] = np.select(conditions, con_choices, default = df_mean_impute['year7_earnings'])

#Check the imputation results
df_mean_impute

In [ ]:
#Multiple imputation

#Save the columns we need in a new DataFrame
df_multi_impute = df_ba_aa[['year7_earnings','gender', 'urm_status', 'high_completion', 'high_completion_label']]

#Get dummy variables
df_multi_impute = pd.get_dummies(df_multi_impute)

#Rename column
df_multi_impute = df_multi_impute.rename(columns = {'urm_status_non-URM' : 'urm_status_non'})

# Identify the column we want to impute
col_for_impute = ['year7_earnings']

# Define the number of imputation
num_impute = 50

#Define an empty DataFrame to save the results
imputed_datasets = []

# Define a seed to get consistent results
np.random.seed(5)

# Using MICE for multiple imputations
mice_impute = mice.MICEData(df_multi_impute)

for i in range(num_impute):
    mice_impute.update_all()
    temp_data = mice_impute.data.copy()
    imputed_datasets.append(temp_data)
    
#Save the final imputed data
df_final_multi_impute = temp_data

#Convert dummy columns back to category column for merging data
df_final_multi_impute.loc[:, 'high_completion_label'] = ''
df_final_multi_impute.loc[df_final_multi_impute['high_completion_label_Associate'] == 1, 'high_completion_label'] = 'Associate'
df_final_multi_impute.loc[df_final_multi_impute['high_completion_label_Bachelor'] == 1, 'high_completion_label'] = 'Bachelor'

#Check the data
df_final_multi_impute

In [ ]:
#Put the original data and imputed data into one DataFrame
missing_df = df_mean_impute[['high_completion_label', 'year7_earnings']]
missing_df.loc[:, 'cat'] = 'Data with Missing Values'

uncon_df = df_mean_impute[['high_completion_label', 'year7_earnings_uncon']]
uncon_df = uncon_df.rename(columns = {'year7_earnings_uncon' : 'year7_earnings'})
uncon_df.loc[:, 'cat'] = 'Unconditional Mean'

con_df = df_mean_impute[['high_completion_label', 'year7_earnings_con']]
con_df = con_df.rename(columns = {'year7_earnings_con' : 'year7_earnings'})
con_df.loc[:, 'cat'] = 'Conditional Mean'

mi_df = df_final_multi_impute[['high_completion_label', 'year7_earnings']]
mi_df.loc[:, 'cat'] = 'Multiple Imputation'

df = pd.concat([missing_df, uncon_df, con_df, mi_df]).sort_values(['high_completion_label', 'cat'])

#Check the data
df.head()

In [ ]:
#Imputed results
df.groupby(['high_completion_label', 'cat'])['year7_earnings'].agg(['mean']).reset_index()

In [ ]:
#Create a bar plot to show results from different imputation methods

# Set plot size
# The first number is the width and the second number is the height.
fig, ax = plt.subplots(figsize = (8, 5))

#Create the bar plot
b = sns.barplot(x = df['cat'], 
                y = df['year7_earnings'], 
                hue = df['high_completion_label'],
                order = ['Data with Missing Values', 'Unconditional Mean', 'Conditional Mean', 'Multiple Imputation'])

# Change Y-axis and X-axis labels
# You can change the label size in `fontsize = `
ax.set_ylabel('Year7 Earnings, Imputed', fontsize = 12)
ax.set_xlabel('Category', fontsize = 12)
#Adjust y axis range
ax.set_ylim([0, 55000])

# Y-axis tick labels: Change the format of earnings displayed on Y-axis
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

# Data source
# The first two numbers indicate the coordinate (or (x,y) position) of the text
# You may need to adjust the numbers a few times to place it in an ideal place
plt.figtext(0.65, 0, 'Data Source: Syntucky Data')

# X-axis tick labels: 
# We can define x tick labels, such as using capital letters for the first letter of each word
# Make sure the order of your labels are consistent with the order you defined in sns.barplot()
ax.set_xticklabels(['Data with Missing Values', 'Unconditional Mean', 'Conditional Mean', 'Multiple Imputation'], 
                   size = 10)

#Adjust legend
ax.legend(title = 'Highest Degree Level')

#Add a title
plt.title('Imputed Year 7 Earnings From Different Imputation Methods' )

# Show the plot
plt.show()

### 4.4. Additional Recommendations

What else can we do to deal with the missingness in our data? **First, we can use additional data sources to fill in missing data whenever possible.** For example, linking with other states' UI wage records and QCEW allows us to get out-of-state employment and earnings. For another example, if we cannot link with other states' data, we may use data with larger coverage, such as Bureau of Motor Vehicle (BMV) data, to check whether a person moved out of state, or National Student Clearinghouse (NSC) data, to check whether a student enrolled in private or out-of-state schools. Of course, additional data sources may have their own limitations: the BMV data does not cover people who do not have driver's licences or state IDs and the NSC data does not cover every college in the US.

**Second, when additional data sources are not available to us, we should assess the data generating process and sources of missingness. Then choose the appropriate imputation methods accordingly.** Is the missing rate particularly high for students from some subgroups? How likely is the data missing at random vs. missing not at random? When compared to other data sources, how do the results based on our data differ?

**Finally, we should document data missingness carefully when reporting the outcomes.** This helps our audience understand and interpret our results properly. We should carefully discuss the coverage and potential missingness of the data in the technical document. We should also report missing rates in key fields, such as the year 7 missing earnings rates by degree type in the above example. With the rapid development in data collection methods, some data missingness issues may be addressed in the future. 